In [6]:
from typing import Optional
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import cfbd
from pprint import pprint
import requests
from enum import Enum

In [7]:
class StatTypes(Enum):
  """
  Stat IDs from api.collegefootballdata.com/plays/stats/types
  """
  INCOMPLETION = 1
  TARGET = 2
  PASS_BREAKUP = 3
  COMPLETION = 4
  RECEPTION = 5
  TACKLE = 6
  RUSH = 7
  FUMBLE = 8
  FUMBLE_FORCED = 9
  FUMBLE_RECOVERED = 10
  SACK_TAKEN = 11
  SACK = 12
  KICKOFF = 13
  ONSIDE_KICK = 14
  KICKOFF_RETURN = 15
  PUNT = 16
  PUNT_BLOCK = 17
  FG_ATTEMPT_BLOCKED = 18
  FIELD_GOAL_BLOCK = 19
  INTERCEPTION_THROWN = 20
  INTERCEPTION = 21
  TOUCHDOWN = 22
  FIELD_GOAL_ATTEMPT = 23
  FIELD_GOAL_MADE = 24
  FIELD_GOAL_MISSED = 25

In [8]:
from typing import Union


class CFBDClient:
  hostname: str
  headers: dict[str, str]
  
  def __init__(self, hostname: str, bearer_token: str):
    self.hostname = hostname
    self.headers = dict()
    self.headers['Authorization'] = f'Bearer {bearer_token}'

  def get(
    self,
    subpath: str,
    headers: Optional[dict[str, str]] = None,
    params: Optional[dict[str, any]] = None,
    as_df: bool = False
  ) -> Union[dict, pd.DataFrame]:
    if headers is None:
      headers = {}
    headers.update(self.headers)

    response = requests.get(
      url=f"{self.hostname}/{subpath}",
      headers=headers,
      params=params  # <-- Pass query parameters here
    )
    if not response.ok:
      raise requests.exceptions.RequestException(
        f"{response.status_code}: {response.text}",
        response=response
      )
    json_data = response.json()
    return json_data if not as_df else self.response_as_df(json_data)
  
  @staticmethod
  def response_as_df(response: Union[dict, list[dict]]) -> pd.DataFrame:
    """
    Convert API JSON response (dict or list of dicts) into a Pandas DataFrame.
    """
    if isinstance(response, dict):
        # If dict has a single top-level key pointing to list of dicts
        # or just a flat dict
        if all(isinstance(v, list) for v in response.values()) and len(response) == 1:
            response = next(iter(response.values()))
        else:
            response = [response]
    return pd.DataFrame(response)


In [9]:
load_dotenv()

client = CFBDClient(
  hostname = 'https://api.collegefootballdata.com/',
  bearer_token = os.getenv('CFBDATA_API_KEY')
)

In [17]:
makes = client.get('plays/stats', params={
  'year': 2024,
  'week': 1,
  'statTypeId': StatTypes.FIELD_GOAL_MADE.value,
  'classification': 'fbs',
}, as_df=True)

In [18]:
attempts = client.get('plays/stats', params={
  'year': 2024,
  'week': 1,
  'statTypeId': StatTypes.FIELD_GOAL_ATTEMPT.value,
  'classification': 'fbs',
}, as_df=True)

In [19]:
len(makes), len(attempts)

(376, 518)